In [1]:
# from geopy.geocoders import Nominatim
from geopy import *
import geopy.distance
import pandas as pd
import numpy as np
import math
import itertools
import re
import Levenshtein

In [2]:
# TODO: what about voltage level?
# TODO: somehow map statuses to some sort of enum
# TODO: map column names to fixed names (see example csv)

#### 2020 ####
column_semantics_2020 = {
    'Investment number': 'investment_id',
    'Commissioning Year': 'commissioning_year',
    'Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction': 'status',
    'Type of Element': 'asset_type',
    'Substation From': 'substation_1',
    'Substation To': 'substation_2',
    'Technology': 'current_type',
    'Total route length (km)': 'specified_length_km'
}

status_map_2020 = {
    1: 1, # under consideration
    2: 2, # planning, not permitting
    3: 3, # in permitting
    4: 4  # under construction
}
params_2020 = {
    'excel': '2020/transmission.xlsx',
    'sheet': 'Trans.Investments',
    'header_row': 1,
    'status_column': 'Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction',
    'status_map': status_map_2020,
    'column_semantics': column_semantics_2020
}

In [3]:
#### 2018 ####
column_semantics_2018 = {
    'Investment ID': 'investment_id',
    'ExpectedCommissioningYear': 'commissioning_year',
    'Status': 'status',
    'ElementsType': 'asset_type',
    'From': 'substation_1',
    'To': 'substation_2',
    'TechnologyType': 'current_type',
    'TotalRouteLength (km)': 'specified_length_km'
}

status_map_2018 = {
    'under consideration': 1,
    'planned but not yet permitting': 2,
    'permitting': 3,
    'under construction': 4
}

params_2018 = {
    'excel': r'2018/TYNDP_2018_Project_List.xlsx',
    'sheet': 'Sheet1',
    'header_row': 0,
    'status_column': 'Status',
    'status_map': status_map_2018,
    'column_semantics': column_semantics_2018
}

In [4]:
params = params_2020

In [5]:
excel = params['excel']
sheet = params['sheet']

column_semantics = params['column_semantics']

wanted_columns = column_semantics.keys()
status_column  = [k for (k,v) in column_semantics.items() if v == 'status'][0]
status_map = params['status_map']
header_row = params['header_row']

wanted = pd.read_excel(excel, sheet_name=sheet, header=header_row)[wanted_columns]

# map columns to specified names (-> consistency & semantics)
wanted.columns = [column_semantics[c] for c in wanted.columns]

if wanted['status'].dtype == pd.StringDtype:
    wanted['status'] = wanted['status'].str.lower()

# only keep rows whose status is specified in status_map as a key
wanted = wanted.loc[wanted['status'].isin(status_map.keys())]

# replace status with numerical values as specified in status_map
wanted = wanted.replace({'status': status_map})


# only choose those in permitting or under construction
wanted = wanted.loc[wanted['status'].astype(int) >= 3]

wanted.head()

,investment_id,commissioning_year,status,asset_type,substation_1,substation_2,current_type,specified_length_km
1,4,2022,3,ACTransmissionLine,V.Minho (by Ribeira de Pena),Feira (by Ribeira de Pena),AC,131.00
2,474,2021,3,OnshoreSubstation,Ribeira de Pena (PT),-,AC,0.00
3,18,2022,3,ACTransmissionLine,Beariz (ES),Fontefria (ES),AC,30.00
4,496,2022,3,ACTransmissionLine,Fontefria (ES),Vila Nova de Famalicão (PT) (By Ponte de Lima),AC,140.21
5,498,2022,3,Transformer,Fontefria (ES),REE,AC,0.00


In [7]:
# TODO: somehow make this work for every case

#### 2020 ####
# ac_lines  = wanted.query("`Type of Element` == 'ACTransmissionLine'")
# dc_lines  = wanted.query("`Type of Element` == 'DCTransmissionLine'")
# on_subst  = wanted.query("`Type of Element` == 'OnshoreSubstation'")
# off_subst = wanted.query("`Type of Element` == 'OffshoreSubstation'")
# 
# lines = ac_lines.append(dc_lines)
lines = wanted.loc[wanted['asset_type'] == 'Overhead Line']

if params == params_2020:
    ac_lines  = wanted.query("asset_type == 'ACTransmissionLine'")
    dc_lines  = wanted.query("asset_type == 'DCTransmissionLine'")
    on_subst  = wanted.query("asset_type == 'OnshoreSubstation'")
    off_subst = wanted.query("asset_type == 'OffshoreSubstation'")

    new_subst = set(on_subst['substation_1']).union(on_subst['substation_2'])
    lines = ac_lines.append(dc_lines)
elif params == params_2018:
    lines = wanted.loc[wanted['asset_type'] == 'Overhead Line']
    new_subst = set(wanted.loc[wanted['asset_type'] == 'Substation']['substation_1'])

lines  = lines.query("substation_1 not in @new_subst")
lines  = lines.query("substation_2 not in @new_subst")

# Use bus names from buses.csv (v0.1.0)
See https://github.com/PyPSA/pypsa-eur/blob/v0.1.0rc/data/entsoegridkit/buses.csv. Data is from 2017 (newer gridkit extracts do not contain 'tags' with substation names).

In [8]:
buses_file = 'buses_v0.1.0.csv'

# see base_network.py in PyPSA-Eur repository
buses = (pd.read_csv(buses_file, quotechar="'",
                     true_values='t', false_values='f',
                     dtype=dict(bus_id="str"))
        .set_index("bus_id")
        .drop(['station_id'], axis=1)
        .rename(columns=dict(voltage='v_nom')))

In [9]:
buses = buses.query('tags.notnull()', engine='python')
buses = buses.query("symbol == 'Substation'")

# Extract 'name_eng' and 'country' from tags in  buses

In [10]:
split_regex = r'("\w+"=>"[^"]*"),' # Form: 'key => value, key => value, ...'

tag_regex   = r'"(?P<key>\w+)"=>"(?P<value>[^"]*)"' # Form: 'key => value'
tag_pattern = re.compile(tag_regex)

rows = []

for index, row in buses.iterrows():
    name    = ''
    country = ''
    x = row['x']
    y = row['y']
    
    tags_string = row['tags']
    
    tags = re.split(split_regex, tags_string)
    
    # Remove whitespaces at front and end, remove None values
    tags = [s.strip() for s in tags]
    tags = list(filter(None, tags))
    
    for tag in tags:
        m = tag_pattern.match(tag)
            
        if m is None:
            print(tag)
            
        # see group names in tag_regex
        key   = m.group('key')
        value = m.group('value')
        
        if key == 'name_eng':
            name = value.strip()
        elif key == 'country':
            country = value.strip()
    
    if name == 'unknown' or not name:
        continue
        
    rows.append((name, country, x, y))

In [11]:
curated_buses = pd.DataFrame.from_records(rows, columns=['name', 'country', 'x', 'y'])

## Remove duplicate rows

In [12]:
curated_buses = curated_buses.loc[~curated_buses.duplicated()]

## There are substations which share the same name but have different coordinates
- large deviation between coordinates => substations are most likely in different countries 
    - BUT: it does occur that different places in the same country get the same name
- small deviation between coordinates => reference to same substation (error in gridextract?)

In [13]:
# TODO: added 'NI' although Northern Ireland probably appears in PyPSA as 'GB'. Find a better solution.
pypsa_countries = ['AL', 'AT', 'BA', 'BE', 'BG', 'CH', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'ME', 'MK', 'NI', 'NL', 'NO', 'PL', 'PT', 'RO', 'RS', 'SE', 'SI', 'SK']

### List of all duplicates

In [14]:
duplicated = curated_buses.loc[curated_buses.name.duplicated()]
duplicated = duplicated.query("country in @pypsa_countries")

# for name in duplicated.name.unique():
#     print(name)
#     for index, row in curated_buses.query('name == @name').iterrows():
#         print(f"({row['x']}, {row['y']}), {row['country']}")
#     print('----')

### Same name and country, large deviations

In [15]:
curated_buses.query("name == 'Yuzhnaya'")

,name,country,x,y
2720,Yuzhnaya,RU,44.817352,48.155093
3851,Yuzhnaya,RU,50.674438,52.002638
3905,Yuzhnaya,RU,36.268616,51.642737
3927,Yuzhnaya,RU,38.685608,51.843414
5378,Yuzhnaya,RU,59.824677,56.576128


### Same name, different country, large deviation

In [16]:
curated_buses.query("name == 'Saida'")

,name,country,x,y
617,Saida,LB,35.400696,33.587167
833,Saida,DZ,0.146942,34.908458


In [17]:
curated_buses.query("name == 'Titan'")

,name,country,x,y
1986,Titan,AL,19.786377,41.619549
2825,Titan,UA,33.767853,46.195993
5825,Titan,RU,34.026031,67.451763


## (TODO) Add new substations

In [18]:
on_subst

# extract country if it matches regex
# otherwise, np.NAN

,investment_id,commissioning_year,status,asset_type,substation_1,substation_2,current_type,specified_length_km
2,474,2021,3,OnshoreSubstation,Ribeira de Pena (PT),-,AC,0.0
6,499,2022,3,OnshoreSubstation,Beariz (ES),Beariz (ES),AC,0.0
7,500,2022,3,OnshoreSubstation,Ponte de Lima (PT),Ponte de Lima (PT),AC,0.0
58,715,2023,4,OnshoreSubstation,Stalpu (RO),Stalpu (RO),AC,0.0
67,701,2025,4,OnshoreSubstation,Resita (RO),Resita (RO),AC,0.0
68,705,2025,3,OnshoreSubstation,Timisoara (RO),Timisoara (RO),AC,0.0
109,1711,2023,4,OnshoreSubstation,Kocin (CZ),NaN,AC,0.0
110,1712,2021,4,OnshoreSubstation,Vitkov (CZ),NaN,AC,0.0
122,631,2024,3,OnshoreSubstation,Bajina Basta (RS),Bajina Basta (RS),AC,0.0
125,1528,2020,4,OnshoreSubstation,SS Kraljevo,SS Kraljevo,AC,0.0


## Remove '(\<Country Code\>) ' from tyndp substation name strings, add new column instead
Otherwise, this could negatively impact the Levenshtein distance.

In [19]:
subst_regex   = r'(?P<place>.+)\s?[\[(](?P<country>\w{2})[)\]]' # Form: 'Glorenza (IT)'
subst_pattern = re.compile(subst_regex)

# TODO: does it make sense to "throw away" information here?
# use this if other pattern does not match to remove comments in parentheses
# e.g. 'Molai (through Sklavouna Terminal)'
alt_regex   = r'(?P<place>.+)\s?[\[(].*[)\]]'
alt_pattern = re.compile(alt_regex)

fr_names     = []
fr_countries = []
to_names     = []
to_countries = []

for index, row in lines.iterrows():    
    fr = row['substation_1']
    to = row['substation_2']
    
    # default values if regex does not match
    fr_name = fr
    to_name = to    
    fr_country = np.NAN
    to_country = np.NAN
    
    fr_match = subst_pattern.match(fr)
    to_match = subst_pattern.match(to)
    
    if fr_match:
        fr_name    = fr_match.group('place').strip()
        fr_country = fr_match.group('country').strip()
    else:
        fr_alt_match = alt_pattern.match(fr)
        if fr_alt_match:
            fr_name = fr_alt_match.group('place')
        
    if to_match:
        to_name    = to_match.group('place').strip()
        to_country = to_match.group('country').strip()
    else:        
        to_alt_match = alt_pattern.match(to)
        if to_alt_match:
            to_name = to_alt_match.group('place')
    
    fr_names.append(fr_name)
    fr_countries.append(fr_country)
    to_names.append(to_name)
    to_countries.append(to_country)

In [20]:
lines['substation_1'] = fr_names
lines['substation_2'] = to_names
lines['country_1'] = fr_countries
lines['country_2'] = to_countries
lines

,investment_id,commissioning_year,status,asset_type,substation_1,substation_2,current_type,specified_length_km,country_1,country_2
1,4,2022,3,ACTransmissionLine,V.Minho,Feira,AC,131.00,NaN,NaN
4,496,2022,3,ACTransmissionLine,Fontefria,Vila Nova de Famalicão,AC,140.21,ES,PT
9,60,2022,4,ACTransmissionLine,Avelin/Mastaing,Horta,AC,80.00,FR,BE
10,614,2023,4,ACTransmissionLine,Nauders,Glorenza,AC,26.00,AT,IT
13,90,2023,3,ACTransmissionLine,Calenzano,Colunga,AC,80.00,IT,IT
...,...,...,...,...,...,...,...,...,...,...
99,1014,2024,3,DCTransmissionLine,Verderio,Bonaduz,DC,165.00,NaN,NaN
139,664,2026,3,DCTransmissionLine,"Brunsbüttel, Wilster","Großgartach, Grafenrheinfeld",DC,700.00,NaN,NaN
161,660,2024,3,DCTransmissionLine,Osterath,Philippsburg,DC,340.00,DE,DE
203,1458,2024,3,DCTransmissionLine,Codrongianos - Lucciana,Suvereto,DC,300.00,NaN,NaN


## create mapping from all unique tyndp substation names to substation names from 'buses'

In [21]:
tyndp_subs   = set(lines['substation_1']).union(set(lines['substation_2']))
tyndp_to_bus = {}

for tyndp in tyndp_subs:
    buses_subs = curated_buses.name.values
    
    closest = min([(bus, Levenshtein.distance(bus.lower(), tyndp.lower())) for bus in buses_subs], key=lambda t: t[1])[0]
    print()
    
    tyndp_to_bus[tyndp] = closest

In [22]:
# a, b = 'Turleenan', 'Guillena'
# a, b = 'Pyhanselka', 'Pyhänselkä'
# a, b = 'Tuomela B', 'Tudela'
# a, b =  'Heviz (HU) \\ Zerjavinec', 'Žerjavinec'
# Levenshtein.distance(a.lower(), b.lower())

In [23]:
# tyndp_to_bus

### Helper functions: Out of all possible pairs of locations from two lists, get the pair whose distance is closest to the specified (line) length
Deals with problem of multiple places in same country sharing a name.

In [24]:
# TODO: keep name!
def extract_coords(rows):
    coordinates = []
    for _, row in rows.iterrows():
        coordinates.append((row['x'], row['y']))
    return coordinates

In [25]:
def match_pair_with_length(s1_rows, s2_rows, length):
    s1_coords = extract_coords(s1_rows)
    s2_coords = extract_coords(s2_rows)
    
    combinations  = list(itertools.product(s1_coords, s2_coords))
    with_distance = [(a, b, geopy.distance.distance(a,b).km) for (a,b) in combinations]
    
    best_match = min(with_distance, key=lambda t: abs(length - t[2]))
    return best_match

# Match start- and endpoints of lines to substations from buses.csv

In [26]:
fr_to_tuples  = {}
# TODO: write out more than indices and use e.g. match for substation 1
error_rows = {}

for index, row in lines.iterrows():
    # Get closest name match based on Levenshtein distance for start- and endpoints of line
    fr = row['substation_1']
    to = row['substation_2']
    
    fr_country = row['country_1']
    to_country = row['country_2']
            
    s1 = tyndp_to_bus[fr]
    s2 = tyndp_to_bus[to]
    
    # Extract respective rows in buses to determine coordinates
    buses_s1 = curated_buses.loc[curated_buses.name == s1]
    buses_s2 = curated_buses.loc[curated_buses.name == s2]
    
    # If we were able to extract country from name, restrict chosen rows to this country.
    if not pd.isna(fr_country):
        buses_s1 = buses_s1.loc[buses_s1['country'] == fr_country]
    if not pd.isna(to_country):
        buses_s2 = buses_s2.loc[buses_s2['country'] == to_country]
    
    if buses_s1.empty or buses_s2.empty:
        error_rows[index] = row
        continue
    
    # Choose pair which matches length best
    length = row['specified_length_km']
    (x1, y1), (x2, y2), coord_dist = match_pair_with_length(buses_s1, buses_s2, length)
        
    tpl = (s1, x1, y1, s2, x2, y2, coord_dist)
    
    # TODO: how to choose an appropriate tolerance?
    if math.isclose(coord_dist, length, rel_tol=0.45):
        error_rows[index] = row
    else:
        fr_to_tuples[index] = tpl

In [27]:
coordinates = pd.DataFrame(index=fr_to_tuples.keys(), data=fr_to_tuples.values(), columns=['s1', 'x1', 'y1', 's2', 'x2', 'y2', 'coord_dist'])

result = lines.copy()
result = result.join(coordinates)

# TODO: why only 26% for 2020? Check beginning of notebook.
percentage = coordinates.index.size / lines.index.size
print(f'{percentage * 100}% of lines are probably correct.')

# print('Lines where we probably found the correct coordinates:')
# result.loc[~result.s1.isna()]

26.08695652173913% of lines are probably correct.


In [28]:
error_lines = result.loc[result.s1.isna()]
error_subst = set(error_lines['substation_1']).union(error_lines['substation_2'])

# print('')
# {(k,tyndp_to_bus[k]) for k in error_subst}

# Determine coordinates using geopy

In [29]:
def match_pair_with_length_geopy(s1_locations, s2_locations, length):
    s1_first_name = s1_locations[0][0]
    s2_first_name = s2_locations[0][0]

    # Only take locations which at least include name of the first location in list (assumption: best name-based match).
    s1_locations = [l for l in s1_locations if s1_first_name in l[0]]
    s2_locations = [l for l in s2_locations if s2_first_name in l[0]]

    return match_coord_pairs_with_length(s1_locations, s2_locations, length)

In [30]:
def lat_lon(loc):
    return (loc.latitude, loc.longitude)

def match_coord_pairs_with_length(s1_coords, s2_coords, length):
    combinations  = list(itertools.product(s1_coords, s2_coords))
    with_distance = [(a, b, geopy.distance.distance(lat_lon(a),lat_lon(b)).km) for (a,b) in combinations]
    
    best_match = min(with_distance, key=lambda t: abs(length - t[2]))
    return best_match

In [31]:
# locator = Nominatim(user_agent='esm_group')
# geocode = RateLimiter(locator.geocode, min_delay_seconds=0.01)
locator = AlgoliaPlaces(user_agent='esm_group')
geocode = locator.geocode

In [32]:
fr_to_tuples_geopy = {}
error_tuples_geopy = {}

for index, row in error_lines.iterrows():
    fr   = row['substation_1']
    to   = row['substation_2']
    dist = row['specified_length_km']

    fr_country = row['country_1']
    to_country = row['country_2']

    # TODO: is it possible to get several matching locations?
    fr_locs = geocode(fr, exactly_one=False) if pd.isna(fr_country) else geocode(fr, exactly_one=False, countries=[fr_country])
    to_locs = geocode(to, exactly_one=False) if pd.isna(to_country) else geocode(to, exactly_one=False, countries=[to_country])
    
    if fr_locs is None or to_locs is None:
        continue
        
    (s1, (x1, y1)), (s2, (x2, y2)), coord_dist = match_pair_with_length_geopy(fr_locs, to_locs, dist)
    tpl = (s1, x1, y1, s2, x2, y2, coord_dist)

    if not math.isclose(coord_dist, dist, rel_tol=0.45):
        error_tuples_geopy[index] = tpl
    else:
        fr_to_tuples_geopy[index] = tpl

In [33]:
coordinates_geopy = pd.DataFrame(index=fr_to_tuples_geopy.keys(), data=fr_to_tuples_geopy.values(), columns=['s1', 'x1', 'y1', 's2', 'x2', 'y2', 'coord_dist'])

coordinates = coordinates.append(coordinates_geopy)

result = lines.copy()
result = result.join(coordinates)

percentage = coordinates.index.size / lines.index.size
print(f'{percentage * 100}% of lines are probably correct.')

print('Lines where we probably found the correct coordinates:')
result.loc[~result.s1.isna()]

76.08695652173914% of lines are probably correct.
Lines where we probably found the correct coordinates:


,investment_id,commissioning_year,status,asset_type,substation_1,substation_2,current_type,specified_length_km,country_1,country_2,s1,x1,y1,s2,x2,y2,coord_dist
4,496,2022,3,ACTransmissionLine,Fontefria,Vila Nova de Famalicão,AC,140.21,ES,PT,Fontefría,42.476200,-7.789700,Vila Nova de Famalicão,41.407900,-8.519800,133.210461
9,60,2022,4,ACTransmissionLine,Avelin/Mastaing,Horta,AC,80.00,FR,BE,Avelin,50.539700,3.082220,Galerie Horta - Hortagalerij,50.845400,4.356700,96.260838
10,614,2023,4,ACTransmissionLine,Nauders,Glorenza,AC,26.00,AT,IT,Nauders,46.891700,10.502600,Glurns - Glorenza,46.671400,10.553900,24.801352
13,90,2023,3,ACTransmissionLine,Calenzano,Colunga,AC,80.00,IT,IT,Via di Calenzano,43.843200,11.188300,Via Colunga,44.481900,11.441900,73.811310
19,144,2020,4,ACTransmissionLine,Audorf,Kassoe,AC,110.00,DE,DK,Schacht-Audorf,54.313200,9.718530,Kassøvej,55.034200,9.214200,86.603200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,616,2028,3,DCTransmissionLine,Salgareda,Bericevo,DC,0.00,IT,SI,Salgareda,12.516174,45.824971,Beričevo,14.521179,46.039876,223.040381
99,1014,2024,3,DCTransmissionLine,Verderio,Bonaduz,DC,165.00,NaN,NaN,Via Don Ambrogio Verderio,45.512200,9.356120,Bonaduz,46.811300,9.398440,144.437764
161,660,2024,3,DCTransmissionLine,Osterath,Philippsburg,DC,340.00,DE,DE,Osterather Straße,50.960100,6.936370,Philippsburg,51.854800,12.039700,368.694133
203,1458,2024,3,DCTransmissionLine,Codrongianos - Lucciana,Suvereto,DC,300.00,NaN,NaN,Codrongianos,40.656000,8.681400,Suvereto,43.079000,10.678700,316.084152


In [34]:
error_coordinates_geopy = pd.DataFrame(index=error_tuples_geopy.keys(), data=error_tuples_geopy.values(), columns=['s1', 'x1', 'y1', 's2', 'x2', 'y2', 'coord_dist'])


In [35]:
error_coordinates_geopy.join(lines)

,s1,x1,y1,s2,x2,y2,coord_dist,investment_id,commissioning_year,status,asset_type,substation_1,substation_2,current_type,specified_length_km,country_1,country_2
1,Vinhós,41.22490,-7.83841,Feira de Santana,-12.2579,-38.9598,6729.773337,4,2022,3,ACTransmissionLine,V.Minho,Feira,AC,131.0,NaN,NaN
24,Gabčíkovo,47.89510,17.57810,Gönyű,47.7331,17.8279,25.969975,1500,2020,4,ACTransmissionLine,Gabcikovo,Gonyu,AC,110.0,SK,HU
29,Hinkley Point Road,51.17730,-3.09620,Seabank Way,52.7647,0.4066,298.512519,458,2024,3,ACTransmissionLine,Hinkley Point,Seabank,AC,60.0,GB,GB
39,Diemenim,7.04671,-9.48218,Ensenada,-34.8570,-57.9077,6878.848564,1488,2022,4,ACTransmissionLine,Diemen,Ens,AC,75.0,NaN,NaN
40,Ensenada,31.86590,-116.60300,Zwolle,31.6316,-93.6441,2171.430927,1490,2024,3,ACTransmissionLine,Ens,Zwolle,AC,32.0,NaN,NaN
46,Keminmaa,65.80300,24.52090,Pyhäselkä,62.4333,29.9667,459.565277,1710,2024,3,ACTransmissionLine,Keminmaa,Pyhänselkä,AC,156.0,NaN,NaN
201,Liefkenshoek,51.13650,4.75960,Mercatorstraße,50.2902,6.0877,132.895031,604,2025,3,ACTransmissionLine,Liefkenshoek,Mercator,AC,19.0,BE,BE
205,Sankt Peter-Ording,54.30360,8.64138,Sankt Johann am Tauern,47.3585,14.4695,874.234757,1472,2025,4,ACTransmissionLine,St. Peter,Tauern,AC,128.0,NaN,NaN
208,Wullenstetten,48.31640,10.06890,Boder,47.5333,14.3667,332.732017,1476,2025,3,ACTransmissionLine,Wullenstetten,Austrian National border,AC,95.0,DE,AT
226,Pfitsch - Val di Vizze,46.90170,11.46450,Steinach am Brenner,47.0907,11.4665,21.011826,1556,2023,4,ACTransmissionLine,Prati di Vizze,Steinach,AC,139.0,IT,AT


In [36]:
# TODO: plot error tuples and matches separately.